In [37]:
# Source: https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch/#create-custom-coco-dataset
import base64
import IPython
import json
import numpy as np
import os
import random
import requests
from io import BytesIO
from math import trunc
from PIL import Image as PILImage
from PIL import ImageDraw as PILImageDraw

In [38]:
# Load the dataset json
class CocoDataset():
    def __init__(self, annotation_path, image_dir):
        self.annotation_path = annotation_path
        self.image_dir = image_dir
        self.colors = ['blue', 'purple', 'red', 'green', 'orange', 'salmon', 'pink', 'gold',
                        'orchid', 'slateblue', 'limegreen', 'seagreen', 'darkgreen', 'olive',
                        'teal', 'aquamarine', 'steelblue', 'powderblue', 'dodgerblue', 'navy',
                        'magenta', 'sienna', 'maroon']
        
        json_file = open(self.annotation_path)
        self.coco = json.load(json_file)
        json_file.close()
        
        #self.process_info()
        #self.process_licenses()
        self.process_categories()
        self.process_images()
        self.process_segmentations()

    def display_info(self):
        print('Dataset Info:')
        print('=============')
        for key, item in self.info.items():
            print('  {}: {}'.format(key, item))
        
        requirements = [['description', str],
                        ['url', str],
                        ['version', str],
                        ['year', int],
                        ['contributor', str],
                        ['date_created', str]]
        for req, req_type in requirements:
            if req not in self.info:
                print('ERROR: {} is missing'.format(req))
            elif type(self.info[req]) != req_type:
                print('ERROR: {} should be type {}'.format(req, str(req_type)))
        print('')
        
    def display_licenses(self):
        print('Licenses:')
        print('=========')
        
        requirements = [['id', int],
                        ['url', str],
                        ['name', str]]
        for license in self.licenses:
            for key, item in license.items():
                print('  {}: {}'.format(key, item))
            for req, req_type in requirements:
                if req not in license:
                    print('ERROR: {} is missing'.format(req))
                elif type(license[req]) != req_type:
                    print('ERROR: {} should be type {}'.format(req, str(req_type)))
            print('')
        print('')
        
    def display_categories(self):
        print('Categories:')
        print('=========')
        for sc_key, sc_val in self.super_categories.items():
            print('  super_category: {}'.format(sc_key))
            for cat_id in sc_val:
                print('    id {}: {}'.format(cat_id, self.categories[cat_id]['name']))
            print('')
    
    def display_image(self, image_id, show_polys=True, show_bbox=True, show_labels=True, show_crowds=True, use_url=False):
        print('Image:')
        print('======')
        if image_id == 'random':
            image_id = random.choice(list(self.images.keys()))
        
        # Print the image info
        image = self.images[image_id]
        for key, val in image.items():
            print('  {}: {}'.format(key, val))
            
        # Open the image
        if use_url:
            image_path = image['coco_url']
            response = requests.get(image_path)
            image = PILImage.open(BytesIO(response.content))
            
        else:
            print(image)
            image_path = os.path.join(self.image_dir, image['file_name'])
            image = PILImage.open(image_path)
            
        buffered = BytesIO()
        image.save(buffered, format="PNG")
        img_str = "data:image/png;base64, " + base64.b64encode(buffered.getvalue()).decode()
        
        # Calculate the size and adjusted display size
        max_width = 900
        image_width, image_height = image.size
        adjusted_width = min(image_width, max_width)
        adjusted_ratio = adjusted_width / image_width
        adjusted_height = adjusted_ratio * image_height
        
        # Create list of polygons to be drawn
        polygons = {}
        bbox_polygons = {}
        rle_regions = {}
        poly_colors = {}
        labels = {}
        print('  segmentations ({}):'.format(len(self.segmentations[image_id])))
        for i, segm in enumerate(self.segmentations[image_id]):
            polygons_list = []
            if segm['iscrowd'] != 0:
                # Gotta decode the RLE
                px = 0
                x, y = 0, 0
                rle_list = []
                for j, counts in enumerate(segm['segmentation']['counts']):
                    if j % 2 == 0:
                        # Empty pixels
                        px += counts
                    else:
                        # Need to draw on these pixels, since we are drawing in vector form,
                        # we need to draw horizontal lines on the image
                        x_start = trunc(trunc(px / image_height) * adjusted_ratio)
                        y_start = trunc(px % image_height * adjusted_ratio)
                        px += counts
                        x_end = trunc(trunc(px / image_height) * adjusted_ratio)
                        y_end = trunc(px % image_height * adjusted_ratio)
                        if x_end == x_start:
                            # This is only on one line
                            rle_list.append({'x': x_start, 'y': y_start, 'width': 1 , 'height': (y_end - y_start)})
                        if x_end > x_start:
                            # This spans more than one line
                            # Insert top line first
                            rle_list.append({'x': x_start, 'y': y_start, 'width': 1, 'height': (image_height - y_start)})
                            
                            # Insert middle lines if needed
                            lines_spanned = x_end - x_start + 1 # total number of lines spanned
                            full_lines_to_insert = lines_spanned - 2
                            if full_lines_to_insert > 0:
                                full_lines_to_insert = trunc(full_lines_to_insert * adjusted_ratio)
                                rle_list.append({'x': (x_start + 1), 'y': 0, 'width': full_lines_to_insert, 'height': image_height})
                                
                            # Insert bottom line
                            rle_list.append({'x': x_end, 'y': 0, 'width': 1, 'height': y_end})
                if len(rle_list) > 0:
                    rle_regions[segm['id']] = rle_list  
            else:
                # Add the polygon segmentation
                for segmentation_points in segm['segmentation']:
                    segmentation_points = np.multiply(segmentation_points, adjusted_ratio).astype(int)
                    polygons_list.append(str(segmentation_points).lstrip('[').rstrip(']'))

            polygons[segm['id']] = polygons_list

            if i < len(self.colors):
                poly_colors[segm['id']] = self.colors[i]
            else:
                poly_colors[segm['id']] = 'white'
            
            bbox = segm['bbox']
            bbox_points = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1],
                           bbox[0] + bbox[2], bbox[1] + bbox[3], bbox[0], bbox[1] + bbox[3],
                           bbox[0], bbox[1]]
            bbox_points = np.multiply(bbox_points, adjusted_ratio).astype(int)
            bbox_polygons[segm['id']] = str(bbox_points).lstrip('[').rstrip(']')
            
            labels[segm['id']] = (self.categories[segm['category_id']]['name'], (bbox_points[0], bbox_points[1] - 4))
            
            # Print details
            print('    {}:{}:{}'.format(segm['id'], poly_colors[segm['id']], self.categories[segm['category_id']]))

        # Draw segmentation polygons on image
        html = '<div class="container" style="position:relative;">'
        html += '<img src="{}" style="position:relative;top:0px;left:0px;width:{}px;">'.format(img_str, adjusted_width)
        html += '<div class="svgclass"><svg width="{}" height="{}">'.format(adjusted_width, adjusted_height)
        
        if show_polys:
            for seg_id, points_list in polygons.items():
                fill_color = poly_colors[seg_id]
                stroke_color = poly_colors[seg_id]
                for points in points_list:
                    html += '<polygon points="{}" style="fill:{}; stroke:{}; stroke-width:1; fill-opacity:0.5" />'.format(points, fill_color, stroke_color)
        
        if show_crowds:
            for seg_id, rect_list in rle_regions.items():
                fill_color = poly_colors[seg_id]
                stroke_color = poly_colors[seg_id]
                for rect_def in rect_list:
                    x, y = rect_def['x'], rect_def['y']
                    w, h = rect_def['width'], rect_def['height']
                    html += '<rect x="{}" y="{}" width="{}" height="{}" style="fill:{}; stroke:{}; stroke-width:1; fill-opacity:0.5; stroke-opacity:0.5" />'.format(x, y, w, h, fill_color, stroke_color)
            
        if show_bbox:
            for seg_id, points in bbox_polygons.items():
                fill_color = poly_colors[seg_id]
                stroke_color = poly_colors[seg_id]
                html += '<polygon points="{}" style="fill:{}; stroke:{}; stroke-width:1; fill-opacity:0" />'.format(points, fill_color, stroke_color)
                
        if show_labels:
            for seg_id, label in labels.items():
                color = poly_colors[seg_id]
                html += '<text x="{}" y="{}" style="fill:{}; font-size: 12pt;">{}</text>'.format(label[1][0], label[1][1], color, label[0])
                
        html += '</svg></div>'
        html += '</div>'
        html += '<style>'
        html += '.svgclass { position:absolute; top:0px; left:0px;}'
        html += '</style>'
        return html
       
    def process_info(self):
        self.info = self.coco['info']
    
    def process_licenses(self):
        self.licenses = self.coco['licenses']
    
    def process_categories(self):
        self.categories = {}
        self.super_categories = {}
        for category in self.coco['categories']:
            cat_id = category['id']
            super_category = category['supercategory']
            
            # Add category to the categories dict
            if cat_id not in self.categories:
                self.categories[cat_id] = category
            else:
                print("ERROR: Skipping duplicate category id: {}".format(category))

            # Add category to super_categories dict
            if super_category not in self.super_categories:
                self.super_categories[super_category] = {cat_id} # Create a new set with the category id
            else:
                self.super_categories[super_category] |= {cat_id} # Add category id to the set
                
    def process_images(self):
        self.images = {}
        for image in self.coco['images']:
            image_id = image['id']
            if image_id in self.images:
                print("ERROR: Skipping duplicate image id: {}".format(image))
            else:
                self.images[image_id] = image
                
    def process_segmentations(self):
        self.segmentations = {}
        for segmentation in self.coco['annotations']:
            image_id = segmentation['image_id']
            if image_id not in self.segmentations:
                self.segmentations[image_id] = []
            self.segmentations[image_id].append(segmentation)

In [39]:
annotation_path = r'./output/seg.json'

# annotation_path = r'./sample.json'
annotation_path = r'./output/realdata_test.json'
annotation_path = r'./output/simtest.json'
image_dir = r'./dataset/lunar/mock_sim/raw'
image_dir = r'./dataset/lunar/combine'
image_dir = r'./dataset/simtest'

listdir = os.listdir(image_dir)
print(listdir)

coco_dataset = CocoDataset(annotation_path, image_dir)
# coco_dataset.display_info()
# coco_dataset.display_licenses()
coco_dataset.display_categories()

['1000_11536.png', '364_4805.png', '365_4817.png', '380_4995.png', '648_8126.png', '649_8138.png', '650_8150.png', '66_1208.png', '67_1219.png', '68_1231.png', '69_1243.png', '70_1255.png', '71_1267.png', '72_1279.png', '73_1291.png', '74_1303.png', '75_1315.png', '76_1327.png', '77_1340.png', '78_1352.png']
Categories:
  super_category: road
    id 0: road

  super_category: person
    id 1: person

  super_category: car
    id 2: car

  super_category: grass
    id 3: grass

  super_category: building
    id 4: building

  super_category: vegetation
    id 5: vegetation

  super_category: pool
    id 6: pool

  super_category: sky
    id 7: sky

  super_category: road line
    id 8: road line



In [40]:
html = coco_dataset.display_image('random', use_url=False)
IPython.display.HTML(html)

Image:
  file_name: 76_1327.png
  height: 600
  width: 800
  id: 17
{'file_name': '76_1327.png', 'height': 600, 'width': 800, 'id': 17}
  segmentations (37):
    699:blue:{'supercategory': 'sky', 'id': 7, 'name': 'sky'}
    700:purple:{'supercategory': 'grass', 'id': 3, 'name': 'grass'}
    701:red:{'supercategory': 'grass', 'id': 3, 'name': 'grass'}
    702:green:{'supercategory': 'grass', 'id': 3, 'name': 'grass'}
    703:orange:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    704:salmon:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    705:pink:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    706:gold:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    707:orchid:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    708:slateblue:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    709:limegreen:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    710:seagreen:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    711:darkgreen:{'supercategory': 'road', 'id'

In [41]:
html = coco_dataset.display_image('random', use_url=False)
IPython.display.HTML(html)

Image:
  file_name: 648_8126.png
  height: 600
  width: 800
  id: 4
{'file_name': '648_8126.png', 'height': 600, 'width': 800, 'id': 4}
  segmentations (21):
    98:blue:{'supercategory': 'sky', 'id': 7, 'name': 'sky'}
    99:purple:{'supercategory': 'grass', 'id': 3, 'name': 'grass'}
    100:red:{'supercategory': 'grass', 'id': 3, 'name': 'grass'}
    101:green:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    102:orange:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    103:salmon:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    104:pink:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    105:gold:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    106:orchid:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    107:slateblue:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    108:limegreen:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    109:seagreen:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    110:darkgreen:{'supercategory': 'road line', 'id

In [42]:
html = coco_dataset.display_image('random', use_url=False)
IPython.display.HTML(html)

Image:
  file_name: 648_8126.png
  height: 600
  width: 800
  id: 4
{'file_name': '648_8126.png', 'height': 600, 'width': 800, 'id': 4}
  segmentations (21):
    98:blue:{'supercategory': 'sky', 'id': 7, 'name': 'sky'}
    99:purple:{'supercategory': 'grass', 'id': 3, 'name': 'grass'}
    100:red:{'supercategory': 'grass', 'id': 3, 'name': 'grass'}
    101:green:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    102:orange:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    103:salmon:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    104:pink:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    105:gold:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    106:orchid:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    107:slateblue:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    108:limegreen:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    109:seagreen:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    110:darkgreen:{'supercategory': 'road line', 'id

In [43]:
html = coco_dataset.display_image('random', use_url=False)
IPython.display.HTML(html)

Image:
  file_name: 78_1352.png
  height: 600
  width: 800
  id: 19
{'file_name': '78_1352.png', 'height': 600, 'width': 800, 'id': 19}
  segmentations (33):
    760:blue:{'supercategory': 'sky', 'id': 7, 'name': 'sky'}
    761:purple:{'supercategory': 'grass', 'id': 3, 'name': 'grass'}
    762:red:{'supercategory': 'grass', 'id': 3, 'name': 'grass'}
    763:green:{'supercategory': 'grass', 'id': 3, 'name': 'grass'}
    764:orange:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    765:salmon:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    766:pink:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    767:gold:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    768:orchid:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    769:slateblue:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    770:limegreen:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    771:seagreen:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    772:darkgreen:{'supercategory': 'road', 'id'

In [44]:
html = coco_dataset.display_image('random', use_url=False)
IPython.display.HTML(html)

Image:
  file_name: 71_1267.png
  height: 600
  width: 800
  id: 12
{'file_name': '71_1267.png', 'height': 600, 'width': 800, 'id': 12}
  segmentations (65):
    335:blue:{'supercategory': 'sky', 'id': 7, 'name': 'sky'}
    336:purple:{'supercategory': 'grass', 'id': 3, 'name': 'grass'}
    337:red:{'supercategory': 'grass', 'id': 3, 'name': 'grass'}
    338:green:{'supercategory': 'grass', 'id': 3, 'name': 'grass'}
    339:orange:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    340:salmon:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    341:pink:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    342:gold:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    343:orchid:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    344:slateblue:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    345:limegreen:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    346:seagreen:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    347:darkgreen:{'supercategory': 'road', 'id'

In [45]:
html = coco_dataset.display_image('random', use_url=False)
IPython.display.HTML(html)

Image:
  file_name: 70_1255.png
  height: 600
  width: 800
  id: 11
{'file_name': '70_1255.png', 'height': 600, 'width': 800, 'id': 11}
  segmentations (50):
    285:blue:{'supercategory': 'sky', 'id': 7, 'name': 'sky'}
    286:purple:{'supercategory': 'grass', 'id': 3, 'name': 'grass'}
    287:red:{'supercategory': 'grass', 'id': 3, 'name': 'grass'}
    288:green:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    289:orange:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    290:salmon:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    291:pink:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    292:gold:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    293:orchid:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    294:slateblue:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    295:limegreen:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    296:seagreen:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    297:darkgreen:{'supercategory': 'road', 'id': 

In [46]:
html = coco_dataset.display_image('random', use_url=False)
IPython.display.HTML(html)

Image:
  file_name: 69_1243.png
  height: 600
  width: 800
  id: 10
{'file_name': '69_1243.png', 'height': 600, 'width': 800, 'id': 10}
  segmentations (36):
    249:blue:{'supercategory': 'sky', 'id': 7, 'name': 'sky'}
    250:purple:{'supercategory': 'grass', 'id': 3, 'name': 'grass'}
    251:red:{'supercategory': 'grass', 'id': 3, 'name': 'grass'}
    252:green:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    253:orange:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    254:salmon:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    255:pink:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    256:gold:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    257:orchid:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    258:slateblue:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    259:limegreen:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    260:seagreen:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    261:darkgreen:{'supercategory': 'road', 'id': 

In [47]:
html = coco_dataset.display_image('random', use_url=False)
IPython.display.HTML(html)

Image:
  file_name: 1000_11536.png
  height: 600
  width: 800
  id: 0
{'file_name': '1000_11536.png', 'height': 600, 'width': 800, 'id': 0}
  segmentations (19):
    0:blue:{'supercategory': 'sky', 'id': 7, 'name': 'sky'}
    1:purple:{'supercategory': 'grass', 'id': 3, 'name': 'grass'}
    2:red:{'supercategory': 'grass', 'id': 3, 'name': 'grass'}
    3:green:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    4:orange:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    5:salmon:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    6:pink:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    7:gold:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    8:orchid:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    9:slateblue:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    10:limegreen:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    11:seagreen:{'supercategory': 'road', 'id': 0, 'name': 'road'}
    12:darkgreen:{'supercategory': 'road', 'id': 0, 'name': 'road'}
